# xgBoost 파이썬

[`xgBoost`](https://xgboost.readthedocs.io/en/latest/)는 Extreme Gradient Boosting 알고리즘으로 원래 C++로 작성되었으나 캐글 경진대회에서 너무나도 뛰어난 성능을 보여 R을 비롯한 파이썬에서도 API를 통해 활용할 수 있도록 저변이 확대되었다. 물론 자바, 스칼라에서도 사용할 수 있다.

xgBoost를 이해하기 위해서는 기본적으로 지도학습(supervised learning), 의사결정나무(decision tree), 부스팅(boosting)과 함께 앙상블(ensemble)에 대한 이해를 전제로 하고 있다. 자세한 사항은 [앙상블(ensemble) 모형](https://statkclee.github.io/model/model-ensemble.html)을 참조한다.

## xgBoost 설치

xgBoost를 설치하는 방법은 윈도우 환경에서 먼저 `The environment is inconsistent, please check the package plan carefully` 오류를 만날 수 있으니... 먼저.. 환경을 최신 환경으로 맞춘 후에 `xgBoost`를 설치한다.

1. `conda clean --all`, `conda update --all`
1. `conda install anaconda`
1. `conda install -c mndrake xgboost`, `conda install -c anaconda py-xgboost`

참고: [How to install xgboost in Anaconda Python (Windows platform)?](https://stackoverflow.com/questions/35139108/how-to-install-xgboost-in-anaconda-python-windows-platform)

## xgBoost 헬로월드

먼저 xgBoost 헬로월드를 찍어보자. 데이터는 데이터캠프 블로그 [Avinash Navlani(August 14th, 2018), "Predicting Employee Churn in Python", Datacamp](https://www.datacamp.com/community/tutorials/predicting-employee-churn-python)을 참조한다.

In [1]:
# 경고 출력하지 않음 -----------
import warnings
warnings.filterwarnings('ignore')

# 라이브러리와 데이터 가져오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn
import sklearn.metrics as metrics

print('sklearn version: %s' % sklearn.__version__)
print('xgboost version: %s' % xgb.__version__)

hr_df = pd.read_csv("data/HR_comma_sep.csv")
hr_df.head()

sklearn version: 0.21.2
xgboost version: 0.90


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
colnames = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 
       'promotion_last_5years']

X, y = hr_df[colnames], hr_df[['left']]
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=777)

xg_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=777)
xg_clf.fit(X_train, y_train)

pred = xg_clf.predict(X_test)
print(f'정확도: {metrics.accuracy_score(y_test, pred)}')

정확도: 0.9664444444444444


# 파이썬 파이프라인(pipeline)

파이썬 파이프라인은 이름을 갖는 튜플 리스트로 기계학습 알고리즘 제작에 scikit-learn 객체를 튜플 형태 파이프에 담아 흘러보낼 수 있다. 이것이 필요한 이유는 xgBoost를 파이썬 파이프라인에 담아내게 되면 피처 공학(Feature Engineering)을 비롯하여 기계학습에 관련된 다양하 작업 절차 및 예측모형 평가를 자동화할 수 있다는 점에서 엄청 편리하다.

## 파이프라인 헬로월드

`xgBoost` 알고리즘을 파이썬 파이프라인에 실어 보내보자.

In [3]:
# 라이브러리와 데이터 가져오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# 훈련/시험 데이터 분리
hr_df = pd.read_csv("data/HR_comma_sep.csv")

colnames = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 
       'promotion_last_5years']

X, y = hr_df[colnames], hr_df[['left']]
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=777)

# 파이프라인 구축
xgb_pipeline = Pipeline([
    ("xgb_model", xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=777))
])

# 예측 ----------------------------
xgb_pipeline.fit(X, y, xgb_model__eval_metric='auc')
preds = xgb_pipeline.predict(X_test)

print(f'정확도(파이프라인): {metrics.accuracy_score(y_test, preds)}')

정확도(파이프라인): 0.9666666666666667
